This Jupyter notebook will focus on analyzing flight delays through a dashboard  
Dashboard components:  
Monthly average carrier delay by reporting airline for the given year.  
Monthly average weather delay by reporting airline for the given year.   
Monthly average national air system delay by reporting airline for the given year.  
Monthly average security delay by reporting airline for the given year.  
Monthly average late aircraft delay by reporting airline for the given yearar.


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})


In [28]:
# Designing layout for application
app = dash.Dash(__name__)

app.layout =  html.Div(
        children = [
        html.H1('Flight Delay Time Statitics', style={'text-align':'center', 'font-size': 40, 'color':'pink'}),
        html.Div(['Input Year', dcc.Input(id = 'input-year', value = '2010', type='number', style={'height':'50px','font-size': 35}),],
        
        style={'font-size': 40}),
         html.Br(),
         html.Br(),
         html.Div(dcc.Graph(id='carrier-delay')),
         html.Div(dcc.Graph(id='weather-delay')),
         html.Div(dcc.Graph(id='national-air-system-delay')),
         html.Div(dcc.Graph(id='security-delay')),
         html.Div(dcc.Graph(id='aircraft-delay')),
]
    )
#computing the required dataframes
# This function will take airline data and input year as input and will perform computation for creating charts and plots
# This function returns the computed average dataframes for 5 charts we need to make
def compute_info(airline_data,entered_year):
    #select data
    df = airline_data[airline_data['Year']==int(entered_year)]
    #compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


# setting up callback decorator
@app.callback([
              Output(component_id='carrier-delay', component_property='figure'),
              Output(component_id='weather-delay', component_property='figure'),
              Output(component_id='national-air-system-delay', component_property='figure'),
              Output(component_id='security-delay', component_property='figure'),
              Output(component_id='aircraft-delay', component_property='figure')],
              Input(component_id='input-year', component_property='value')
             )

# now defining callback function
def get_graph(entered_year):
    
    # Compute required information for creating graph from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)
            
    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]
# Run the app
if __name__ == '__main__':
    app.run_server(port=89)
